# Introduction

This notebook visualizes the results of the registration analysis carried out in this [notebook: 3_registration](3_registration.ipynb)

NOTE: The only purpose of this notebook is to make it easy to visually inspect the outputs from the previous notebook, it does not write any files to disk

# Imports

In [ ]:
import numpy as np
import scipy as sp
from scipy import signal
import skimage
from skimage import morphology as sk_morph
from skimage import exposure
import os
import json

from IPython import display
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1 import ImageGrid
from tqdm import tqdm

from fam13a import utils, image, register
from multiprocessing import Pool
from functools import partial

# Constants

In [ ]:
PROJ_ROOT = utils.here()
# declare the data input directory
HBEC_ROOT = os.path.join(PROJ_ROOT, 'data', 'processed', 'hbec')
# print list of experiment IDs
print(os.listdir(HBEC_ROOT))

In [ ]:
# choose experiment data to load
EXP_ID = 'N67030-59_8_perc'

# declare the various output directories
PROCESSED_ROOT = os.path.join(HBEC_ROOT, EXP_ID)
MAX_FRAME_ROOT = os.path.join(PROCESSED_ROOT, 'max_frame')
REGISTER_ROOT = os.path.join(PROCESSED_ROOT, 'register')

# find all relevant data files in the data directory 
file_ids = sorted([_d for _d in os.listdir(REGISTER_ROOT) if os.path.isdir(os.path.join(REGISTER_ROOT, _d))])
# print available data file IDs
print(file_ids)

# Load data

In [ ]:
# choose data file to load
_file_id = 'NT_6_1_MMStack_Pos0'
FILE_ROOT = os.path.join(REGISTER_ROOT, _file_id)

# load each of the associated interim/processed files
shifts = np.load(os.path.join(FILE_ROOT, 'shifts.npy'))
max_frame = np.load(os.path.join(MAX_FRAME_ROOT, f'{_file_id}.npy'))
sub_max_frame = np.load(os.path.join(FILE_ROOT, 'max_frame.npy'))
sub_movement_mask = np.load(os.path.join(FILE_ROOT, 'mask.npy'))
validation_angles = np.load(os.path.join(FILE_ROOT, 'validation_angles.npy'))
with open(os.path.join(FILE_ROOT, 'params.json'), 'r') as params_file:
    params = json.load(params_file)

# Process

we explicitly set the min and max values, of the colour map, for the vector magnitudes. This is done so the vector field plots of different videos can be compared visually, otherwise matplotlib automatically uses the min and max values present in the image to set the range of the colour map

In [ ]:
min_mag, max_mag = register.calculate_min_max_speed(params['ptrn_size'], params['region_size'])

In [ ]:
# calculate average velocity field from shifts array
velocity_fields = register.calculate_mean_velocity_field(shifts)
norm_shifts = velocity_fields['normalised_velocity']
mags = velocity_fields['speed']

In [ ]:
# create validation angle mask with threshold of 30 degrees
validation_angle_mask = (validation_angles < 30.)

In [ ]:
# mask out vectors not in the moving region segmentation mask 
masked_norm_shifts = norm_shifts.copy()
masked_norm_shifts[~sub_movement_mask, :] = 0
masked_norm_shifts[~validation_angle_mask, :] = 0
masked_mags = mags.copy()
masked_mags[~sub_movement_mask] = None
masked_mags[~validation_angle_mask] = None

# Visualize

In [ ]:
# plot the max projection view
plt.figure(figsize=(10, 10))
plt.imshow(max_frame)

In [ ]:
# define the position of box to be zoomed in on in the next cell
slice_ranges = [50, 100, 150, 200]
slices_1 = (slice(slice_ranges[0],slice_ranges[1]), slice(slice_ranges[2],slice_ranges[3]))

# plot full view
plt.figure(figsize=(10, 10))
plt.imshow(sub_max_frame)
plt.quiver(masked_norm_shifts[...,1], masked_norm_shifts[...,0], masked_mags[...], scale=1, scale_units='xy', angles='xy',
          headwidth=4, headlength=4, headaxislength=3, cmap='hot')
plt.clim(min_mag, max_mag)
ax = plt.gca()
rect_1 = patches.Rectangle((slices_1[1].start,slices_1[0].start),
                 slices_1[1].stop - slices_1[1].start,
                 slices_1[0].stop - slices_1[0].start,
                 linewidth=4,
                 edgecolor='cyan',
                 fill = False)
ax.add_patch(rect_1)

In [ ]:
ptrn_size = params["ptrn_size"]
ptrn_size[0]

up_slice_ranges = [sr*ptrn_size[0]+(ptrn_size[0]-1) for sr in slice_ranges]
slices = (slice(*slice_ranges[:2]), slice(*slice_ranges[2:]))
up_slices = (slice(*up_slice_ranges[:2]), slice(*up_slice_ranges[2:]))

tile_mask = np.zeros(ptrn_size).astype(bool)
tile_mask[2,2] = True
tiled_mask = np.tile(tile_mask, (slice_ranges[1] - slice_ranges[0], slice_ranges[3] - slice_ranges[2]))

tiled_vecs = masked_norm_shifts[slices].repeat(ptrn_size[0], axis=0).repeat(ptrn_size[0], axis=1)
tiled_vecs[~tiled_mask] = None

tiled_mags = masked_mags[slices].repeat(ptrn_size[0], axis=0).repeat(ptrn_size[0], axis=1)
tiled_mags[~tiled_mask] = None

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(max_frame[up_slices])
plt.quiver(tiled_vecs[...,1], tiled_vecs[...,0], tiled_mags, scale=0.2, scale_units='xy', angles='xy',
          headwidth=2, headlength=2, headaxislength=2.5, cmap='hot')
plt.clim(min_mag, max_mag)